In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

import os
import sys
proj_path = os.path.join(os.getcwd(), '..', '..')
sys.path.append(proj_path)

from utils.transformations import reusable

In [0]:
df_user = spark.readStream.format("cloudFiles") \
    .option("cloudFiles.format", "parquet") \
    .option("cloudFiles.schemaLocation", "abfss://silver@adityaazureproject.dfs.core.windows.net/DimUser/checkpoint") \
    .option("schemaEvolutionMode", "addNewColumns") \
    .load("abfss://bronze@adityaazureproject.dfs.core.windows.net/DimUser")

In [0]:
display(df_user, checkpointLocation = "abfss://silver@adityaazureproject.dfs.core.windows.net/DimUser/checkpoint")

In [0]:
df_user = df_user.withColumn("user_name", upper(col("user_name")))
display(df_user, checkpointLocation = "abfss://silver@adityaazureproject.dfs.core.windows.net/DimUser/checkpointv2")


In [0]:
df_user_obj = reusable()

df_user = df_user_obj.dropColumns(df_user, ["_rescued_data"])
df_user = df_user.dropDuplicates(['user_id'])

display(df_user, checkpointLocation = "abfss://silver@adityaazureproject.dfs.core.windows.net/DimUser/checkpointv4")

In [0]:
df_user.writeStream.format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", "abfss://silver@adityaazureproject.dfs.core.windows.net/DimUser/checkpointv5") \
    .trigger(once=True) \
    .option("path", "abfss://silver@adityaazureproject.dfs.core.windows.net/DimUser/data") \
    .toTable("spotify_cataa.silver.dimuser")

## **DimArtist**

In [0]:
df_art = spark.readStream.format("cloudFiles") \
    .option("cloudFiles.format", "parquet") \
    .option("cloudFiles.schemaLocation", "abfss://silver@adityaazureproject.dfs.core.windows.net/DimArtist/checkpoint") \
    .option("schemaEvolutionMode", "addNewColumns") \
    .load("abfss://bronze@adityaazureproject.dfs.core.windows.net/DimArtist")

In [0]:
display(df_art, checkpointLocation = "abfss://silver@adityaazureproject.dfs.core.windows.net/DimArtist/checkpointv2")

In [0]:
df_art_obj = reusable()
df_art = df_art_obj.dropColumns(df_art, ["_rescued_data"])
df_art = df_art.dropDuplicates(['artist_id'])
display(df_art, checkpointLocation = "abfss://silver@adityaazureproject.dfs.core.windows.net/DimArtist/checkpointv3")

In [0]:
df_user.writeStream.format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", "abfss://silver@adityaazureproject.dfs.core.windows.net/DimArtist/checkpointv4") \
    .trigger(once=True) \
    .option("path", "abfss://silver@adityaazureproject.dfs.core.windows.net/DimArtist/data") \
    .toTable("spotify_cataa.silver.dimartist")

In [0]:
df_track = spark.readStream.format("cloudFiles") \
    .option("cloudFiles.format", "parquet") \
    .option("cloudFiles.schemaLocation", "abfss://silver@adityaazureproject.dfs.core.windows.net/DimTrack/checkpoint") \
    .option("schemaEvolutionMode", "addNewColumns") \
    .load("abfss://bronze@adityaazureproject.dfs.core.windows.net/DimTrack")

In [0]:
display(df_track, checkpointLocation = "abfss://silver@adityaazureproject.dfs.core.windows.net/DimTrack/checkpointv2")

In [0]:
df_track = df_track.withColumn("durationFlag", when(col("duration_sec") < 150, "low")\
    .when((col("duration_sec") >= 150) & (col("duration_sec") < 300), "medium")\
    .otherwise("high"))
display(df_track, checkpointLocation = "abfss://silver@adityaazureproject.dfs.core.windows.net/DimTrack/checkpointv3")

In [0]:
df_track = reusable().dropColumns(df_track, columns=["_rescued_data"])


In [0]:
df_track.writeStream.format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", "abfss://silver@adityaazureproject.dfs.core.windows.net/DimTrack/checkpointv4") \
    .trigger(once=True) \
    .option("path", "abfss://silver@adityaazureproject.dfs.core.windows.net/DimTrack/data") \
    .toTable("spotify_cataa.silver.dimtrackW")